In [1]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import regex

In [2]:
if sys.platform == 'linux':
    load_dotenv(dotenv_path=Path('.') / '.env.linux')
elif sys.platform == 'win32':
    load_dotenv(dotenv_path=Path('.') / '.env.win')
else:
    raise ValueError('Ваша операционная система не поддерживается')

DATASET_FOLDER = os.getenv('DATASET_FOLDER_PATH', None)

In [3]:
prepared_df = pd.read_csv(os.path.join(DATASET_FOLDER, 'all_recepies_preprocessed.csv')).rename(columns={'Instructions' : 'instructions'})

In [4]:
for i in range(len(prepared_df)):
    prepared_df.loc[i, 'name'] = prepared_df.loc[i, 'name'][0].upper() + prepared_df.loc[i, 'name'][1:]
    prepared_df.loc[i, 'name'] = regex.sub(r'^\n', '', prepared_df.loc[i, 'name'])
    prepared_df.loc[i, 'name'] = regex.sub(r'\n$', '', prepared_df.loc[i, 'name'])

    prepared_df.loc[i, 'instructions'] = regex.sub(r'\r', '', prepared_df.loc[i, 'instructions'])
    prepared_df.loc[i, 'instructions'] = regex.sub(r'^\n', '', prepared_df.loc[i, 'instructions'])
    prepared_df.loc[i, 'instructions'] = regex.sub(r'\n$', '', prepared_df.loc[i, 'instructions'])

    prepared_df.loc[i, 'ingredients'] = regex.sub(r' \.\n', '\n', prepared_df.loc[i, 'ingredients'])
    prepared_df.loc[i, 'ingredients'] = regex.sub(r'^\n', '', prepared_df.loc[i, 'ingredients'])
    prepared_df.loc[i, 'ingredients'] = regex.sub(r'\n$', '', prepared_df.loc[i, 'ingredients'])

In [5]:
prepared_df

,name,instructions,ingredients,composition_list
0,Рассольник классический с перловкой и солеными...,Подготовить указанные ингредиенты для приготов...,Перловка 0.1 стак.\n Соленые огурцы 0.4 шт.\n...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ..."
1,Суп пюре из белокочаной капусты,"Необходимые ингредиенты\nНарезаем лук, морковь...",Капуста белокочанная 50.0 гр.\n Картошка 30.0...,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},..."
2,Постные щи из квашеной капусты,"Честно признаюсь, у меня не было репы на момен...",Капуста квашеная 116.7 гр.\n Мак пищевой 0.5 ...,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'..."
3,Тюря- простой суп быстро и вкусно,"Начинаем мы приготовление тюри с того, что с з...",Квас 0.2 л.\n Лук репчатый 0.2 шт.\n Черный х...,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ..."
4,Фасолевый суп из красной фасоли,Подготовить ингредиенты. Для приготовления суп...,Вода 0.3 л.\n Картошка 0.3 шт.\n Морковь 0.2 ...,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,..."
...,...,...,...,...
27879,Салат с филе тунца и огурцами в азиатской…,1. Для приготовления азиатского соуса смешать ...,Тунец 50.0 гр.\n Огурцы 50.0 гр.\n Пита 0.5 ш...,"[{'Тунец': 50.0, 'unit': 'г'}, {'Огурцы': 50.0..."
27880,Теплый салат с куриной печенью и яблоками,"1. Маленькими ломтиками порезать яблоки, запек...",Куриная печень 120.0 гр.\n Кислые яблоки 30.0...,"[{'Куриная печень': 120.0, 'unit': 'г'}, {'Кис..."
27881,Самый зеленый салат,"1. Чеснок натереть, сыр фета раскрошить.\n2. Я...",Мангольд по вкусу по_вкусу.\n Яблоки «гренни-...,"[{'Мангольд': 'по вкусу', 'unit': 'по вкусу'},..."
27882,Теплый салат с тыквой и брынзой под…,"1. Тыкву нарезаем кубиками, добавляем оливково...",Тыква 100.0 гр.\n Сыр брынза 40.0 гр.\n Кедро...,"[{'Тыква': 100.0, 'unit': 'г'}, {'Сыр брынза':..."


In [6]:
prepared_df.to_csv(os.path.join(DATASET_FOLDER, 'all_recepies_preprocessed_fixed.csv'))

In [7]:
def concat_features_in_one(row):
    return f'''Название рецепта: {row['name']}

Ингридиенты:
{row['ingredients']}

Процесс приготовления:
{row['instructions']}'''

prepared_df['prompt'] = prepared_df.apply(concat_features_in_one, axis=1)

In [8]:
prepared_df.to_csv(os.path.join(DATASET_FOLDER, 'fine_tuning_preprocessed.csv'))